### DIABETES PREDICTION

In [1]:
################################################
# End-to-End Diabetes Machine Learning Pipeline III
################################################


# Bir şekildebir pipeline oluşturduğumuzu düşünelim. Yani modelledik, modeli çıkardık, sakladık. İşiimiz bir noktada bitti, bu
# model nesnesini mesela şu anda takım arkadaşımıza göndderdik. Model development sürecinden sonrs bir modeli canlı sistemlere 
# entegre etmek demek o model nesnesini çağırmak demektşr. Bunu kapsamlı şekilde ele aldığımız diğer programlar var. Fakat o 
# noktalarda da kendi kişisel çalışmalarımızda da çok sinsi bir problem vardır. Saç baş yoldurup bu alan bana göre değil mi 
# acaba diye düşündüren bir problem vardır. Diyelim ki çalışma bitti bunu birisi ile paylaştık. Çalışma ortamını baştan 
# başlattığımızda neticesinde madem bu model nesnesini okuyup kullanabiliyoruz, okumaya çalışalım bakalım diye düşünüyoruz.

# import işlemleri gerçekleştirdik:
import joblib
import pandas as pd

# Veri setini okuduk:
df = pd.read_csv("datasets/diabetes.csv")

# Diyelim ki yeni hastalar geldi. Problem şu hastanemizde yapay zeka ile diyabet tahmini işi yapıyoruz. Model kurduk, sakladık
# her şey başarılı. Yeni kayaıtlar geldi. Diyelim ki csv dosyasına bu kayıtlar düştü. Yüksek ihtimalle bu şekilde olur, bir 
# database'e düşer. Oradan csv olarak çıkarılabilir. Ya da bir uygulamaya düşer. Daha sonra bir database tablosuna düşer ve 
# oradan çekilir. Yani bir şekilde bunların yeni kayıtları barındırdığını düşünelim. 

# Buradan rastgele bir hasta çekelim:
random_user = df.sample(1, random_state=45)

# joblib'den modeli çağıralım:
new_model = joblib.load("voting_clf.pkl")

# Aşağıdaki şekilde yazdığımızda bir boyut hatası alırız. Bunun sebebi modeli kurarken veri setinin altından girip üstünden 
# çıktık ve veri setini değiştirdik. Oradaki featurelar artık farklı featurelar, yeni featurelar türettik. Yeni bir kullanıcı,
# hasta geldi. İyi de o hastanın bilgileri artık bizim eski yapımıza uygun değil ki, bunlar farklı değişkenler, bizim modeli
# kurduğumuz verideki değşkenler farklı değişkenler. Bu veri setinin eski veri setiyle aynı olması dönüştürülmesi lazım. 
# model deployment süreçlerindeki önemli bariyerlerden problemlerden biris budur. Basit ama en başarılı olabilecek modeli 
# oluşturma kaygısının da temel sebeplerinden birisi budur. 40 takla atarak oluşturduğumuz bir feature'ın gerçek hayatta 
# gözlenme freaknsı çok çok düşük olabilir, bir de varsayalım ki feature türetme işlemlerini veri tabanı seviyesinde eda 
# süreçlerinden sonra yapıyoruz. Nihai bir aggrigated feature tablosu oluşturuyoruz tekilleştirilmiş gözlem birimleri adına.
# Dolayısıyla bir de bu şekilde uzun süreçler barındıran bir proje döngüsünde o değişkenlerin üretilmesi veri tabanı yani 
# sunucular seviyesinde de bir problem meşguliyet işlenmesi de bir problem. Hiç gelmeyecek bir feature'ı da üretemeyip bu 
# sefer bu pipeline'ın bu boru hattının veri tabanı basamağında problemlerle karşılaşma ihtimali de bir ihtimaldir. 
# Dolayısıyla burada basitlik vurgusunu yapark bir probleme dikkat çekmeye çalıştık.
new_model.predict(random_user)

C:\Users\aleyy\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Age
- BloodPressure
- DiabetesPedigreeFunction
- Glucose
- Insulin
- ...
Feature names seen at fit time, yet now missing:
- AGE
- BLOODPRESSURE
- DIABETESPEDIGREEFUNCTION
- GLUCOSE
- INSULIN
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 9 features, but KNeighborsClassifier is expecting 16 features as input.

In [2]:
# Şimdi bu sorunun çözmeye çalışalım. Bu veri setini benzer bir şekilde diyabet dataprep modülümüzden (fonksiyonumuzdan) 
# geçirmemiz lazım.

# DİKKAT : Bunu nasıl gerçekleştirbiliriz? from diabetes_pipeline import diabetes_data_prep dersek bizim bu scriptimizin 
# içerisinden sadece bu (diabetes_data_prep) fonksiyonu getiriyor olacak. Şu anda diabetes_data_prep'i çalışmamıza import
# ettik. 

from diabetes_pipeline import diabetes_data_prep

# Bu hatayı çözmek için pipeline kısmında bazı fonksiyonları buraya getirmemiz gerekiyor.

ModuleNotFoundError: No module named 'diabetes_pipeline'

In [10]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    # print(f"Observations: {dataframe.shape[0]}")
    # print(f"Variables: {dataframe.shape[1]}")
    # print(f'cat_cols: {len(cat_cols)}')
    # print(f'num_cols: {len(num_cols)}')
    # print(f'cat_but_car: {len(cat_but_car)}')
    # print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

def diabetes_data_prep(dataframe):
    dataframe.columns = [col.upper() for col in dataframe.columns]

    # Glucose
    dataframe['NEW_GLUCOSE_CAT'] = pd.cut(x=dataframe['GLUCOSE'], bins=[-1, 139, 200], labels=["normal", "prediabetes"])

    # Age
    dataframe.loc[(dataframe['AGE'] < 35), "NEW_AGE_CAT"] = 'young'
    dataframe.loc[(dataframe['AGE'] >= 35) & (dataframe['AGE'] <= 55), "NEW_AGE_CAT"] = 'middleage'
    dataframe.loc[(dataframe['AGE'] > 55), "NEW_AGE_CAT"] = 'old'

    # BMI
    dataframe['NEW_BMI_RANGE'] = pd.cut(x=dataframe['BMI'], bins=[-1, 18.5, 24.9, 29.9, 100],
                                        labels=["underweight", "healty", "overweight", "obese"])

    # BloodPressure
    dataframe['NEW_BLOODPRESSURE'] = pd.cut(x=dataframe['BLOODPRESSURE'], bins=[-1, 79, 89, 123],
                                            labels=["normal", "hs1", "hs2"])

    cat_cols, num_cols, cat_but_car = grab_col_names(dataframe, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    df = one_hot_encoder(dataframe, cat_cols, drop_first=True)

    cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)

    replace_with_thresholds(df, "INSULIN")

    X_scaled = StandardScaler().fit_transform(df[num_cols])
    df[num_cols] = pd.DataFrame(X_scaled, columns=df[num_cols].columns)

    y = df["OUTCOME"]
    X = df.drop(["OUTCOME"], axis=1)

    return X, y

In [13]:
from diabetes_pipeline import diabetes_data_prep

In [15]:
# x ve y'yi çıkaralım. Yeni hastalar veya müşteriler geldi, yeni bir veri var. Bu veriyi prediction işlemi için aynı script'ten
# geçirmek zorundayız. prediction yerine farklı birçok kaynakta bu bölüm karşımıza acoring olarak da gelebilir. Bu modeli 
# kullanıp buradan bir şeyler üretmeye prediction ya da scoring işlemi denmektedir. 
X, y = diabetes_data_prep(df)

# 1 tane örnek seçelim:
random_user = X.sample(1, random_state=50)

# Aynı işlemi tekrarlayalım:
new_model = joblib.load("voting_clf.pkl")

new_model.predict(random_user)

array([0], dtype=int64)

**Anlaşılacağı üzere modeli tekrar kullanmak istediğimizde bu modelin anladığı ve bildiği bir şema var, o şemaya uygun bir şekilde değişken seti göndermemiz gerekiyor. Bundan dolayı buradaki veri ön işleme işlemini tekrar ele aldık ama bunun yeni veri olduğunu varsaydık.**